In [1]:
!pip install antlr4-python3-runtime==4.9.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.1 MB/s eta 0:00:001.2 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=3cc73095827ba2c5bc280b053ae31c220edc5bca4b9e922a17601f90906fe987
  Stored in directory: /Users/nickouellet/Library/Caches/pip/wheels/48/6a/c2/acb58c7afdf57e4cddf5e1513f5a2d62aa8e98f82a00c76d7c
Successfully built antlr4-python3-runtime
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.7.1
    Uninstalling antlr4-python3-runtime-4.7.1:
      Successfully uninstalled antlr4-python3-runtime-4.7.1


In [1]:
from djsql import parse_statement, print_tree

In [22]:
query1 = """

select
	s_name,
	count(*) as numwait
from
	supplier,
	lineitem l1,
	orders,
	nation
where
	s_suppkey = l1.l_suppkey
	and o_orderkey = l1.l_orderkey
	and o_orderstatus = 'F'
	and l1.l_receiptdate > l1.l_commitdate
	and exists (
		select
			*
		from
			lineitem l2
		where
			l2.l_orderkey = l1.l_orderkey
			and l2.l_suppkey <> l1.l_suppkey
	)
	and not exists (
		select
			*
		from
			lineitem l3
		where
			l3.l_orderkey = l1.l_orderkey
			and l3.l_suppkey <> l1.l_suppkey
			and l3.l_receiptdate > l3.l_commitdate
	)
	and s_nationkey = n_nationkey
	and n_name = ':1'
group by
	s_name
order by
	numwait desc,
	s_name
LIMIT 100;
"""

In [23]:
query2="""
WITH sales_by_country AS (
    SELECT country, SUM(amount) AS total_sales
    FROM sales
    GROUP BY country
),
top_countries AS (
    SELECT country, total_sales
    FROM sales_by_country
    ORDER BY total_sales DESC
    LIMIT 3
)
SELECT *
FROM customers
WHERE country IN (
    SELECT country
    FROM top_countries
)
UNION
SELECT *
FROM customers
WHERE country NOT IN (
    SELECT country
    FROM top_countries
)

"""

In [24]:
tree = parse_statement(query1)

In [16]:
print_tree(tree)

|SingleStatementContext
|-StatementDefaultContext
|--QueryContext
|---CtesContext
|----TerminalNodeImpl[WITH]
|----NamedQueryContext
|-----ErrorCapturingIdentifierContext
|------IdentifierContext
|-------UnquotedIdentifierContext
|--------TerminalNodeImpl[sales_by_country]
|------RealIdentContext
|-----TerminalNodeImpl[AS]
|-----TerminalNodeImpl[(]
|-----QueryContext
|------QueryTermDefaultContext
|-------QueryPrimaryDefaultContext
|--------RegularQuerySpecificationContext
|---------SelectClauseContext
|----------TerminalNodeImpl[SELECT]
|----------NamedExpressionSeqContext
|-----------NamedExpressionContext
|------------ExpressionContext
|-------------PredicatedContext
|--------------ValueExpressionDefaultContext
|---------------ColumnReferenceContext
|----------------IdentifierContext
|-----------------UnquotedIdentifierContext
|------------------TerminalNodeImpl[country]
|-----------TerminalNodeImpl[,]
|-----------NamedExpressionContext
|------------ExpressionContext
|-------------P

In [6]:
from djsql import SqlBaseParser as sbp
from dj.sql.parsing import ast
from dj.sql.parsing.backends.exceptions import DJParseException
import antlr4

In [9]:
help(sbp.QueryContext)

Help on class QueryContext in module grammar.generated.SqlBaseParser:

class QueryContext(antlr4.ParserRuleContext.ParserRuleContext)
 |  QueryContext(parser, parent: antlr4.ParserRuleContext.ParserRuleContext = None, invokingState: int = -1)
 |  
 |  Method resolution order:
 |      QueryContext
 |      antlr4.ParserRuleContext.ParserRuleContext
 |      antlr4.RuleContext.RuleContext
 |      antlr4.tree.Tree.RuleNode
 |      antlr4.tree.Tree.ParseTree
 |      antlr4.tree.Tree.SyntaxTree
 |      antlr4.tree.Tree.Tree
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, parser, parent: antlr4.ParserRuleContext.ParserRuleContext = None, invokingState: int = -1)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  accept(self, visitor: antlr4.tree.Tree.ParseTreeVisitor)
 |  
 |  ctes(self)
 |  
 |  enterRule(self, listener: antlr4.tree.Tree.ParseTreeListener)
 |  
 |  exitRule(self, listener: antlr4.tree.Tree.ParseTreeListener)
 |  
 | 

In [ ]:
def parse_query(parse_tree: dict) -> ast.Query:
    """parse a query (ctes+select) statement"""
    if match_keys_subset(parse_tree, {"with", "body", "limit", "order_by"}):
        body = parse_tree["body"]
        if match_keys(body, {"Select"}):
            select = parse_select(body["Select"])
            select.limit = None
            if parse_tree["limit"] is not None:
                limit_value = parse_value(parse_tree["limit"])
                if not isinstance(limit_value, ast.Number):
                    raise DJParseException("limit must be a number")  # pragma: no cover
                select.limit = limit_value
            select.order_by = [
                parse_order(order) for order in parse_tree.get("order_by", [])
            ]


In [25]:
tree.children[0].children[0].ctes()

In [ ]:
#TODO: Add to ast
class SetOp:
    kind: str

In [48]:
from functools import singledispatch

@singledispatch
def visit(ctx):
    raise DJParseException(f"Could not parse {ctx}")
    
@visit.register
def _(ctx: sbp.SingleStatementContext):
    return visit(ctx.statement())

@visit.register
def _(ctx: sbp.StatementDefaultContext):
    return visit(ctx.query())

@visit.register
def _(ctx: sbp.QueryContext):

    
    ctes = []
    if ctes_ctx:=self.ctes():
        ctes = visit(ctes_ctx)
        
    #TODO: orgs move from ast.Select to ast.Query
    if orgs_ctx:=self.queryOrganization():
        orgs = visit(orgs_ctx)
    queryTerm(self)
    
    return ast.Query(
        ctes=ctes,
        select=select,
        set_ops
    )

@visit.register
def _(ctx: sbp.QueryTermContext):
    return select, []

@visit.register
def _(ctx: sbp.CtesContext):
    ...
    
@visit.register
def _(ctx: sbp.QueryOrganizationContext):
    ...

In [26]:
ast.Query?

In [49]:
visit(5)

DJParseException: Could not parse 5